In [4]:
import os

from dotenv import load_dotenv

In [5]:
def llm_call(prompt: str, system_prompt: str = "", model="claude-3-5-sonnet-20241022", allowed_domains=None ) -> str:
    """
    Calls the model with the given prompt and returns the response.

    Args:
        prompt (str): The user prompt to send to the model.
        system_prompt (str, optional): The system prompt to send to the model. Defaults to "".
        model (str, optional): The model to use for the call. Defaults to "claude-3-5-sonnet-20241022".

    Returns:
        str: The response from the language model.
    """
    load_dotenv()
    anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
    # client = Anthropic(api_key=os.environ["ANTHROPIC_API_KEY"])
    client = Anthropic(api_key=anthropic_api_key)
    messages = [
        {"role": "user", "content": prompt}]
    tools = [{
        "type": "web_search_20250305",
        "name": "web_search",
        "max_uses": 5
    }]
    response = client.messages.create(
        model=model,
        max_tokens=4096,
        system=system_prompt,
        messages=messages,
        tools=tools,
        temperature=0

    )
    return response.content[0].text



In [29]:
from anthropic import Anthropic

client = Anthropic()

# def rate_retrieval_w_tool(reference_rate, url):
#     # Create a message with web search enabled

message = client.messages.create(
    model="claude-3-5-sonnet-latest",
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": f"""Find the most recent SOFR rate published by Federal Reserve Bank of New York.
                       Extract only the numerical SOFR values and associated date from the web page.
                       Respond in format 'Date: Value'. Example format:
                        2025-06-12: 4.28"""
        }
    ],
    tools=[
        {
            "name": "web_search",
            "type": "web_search_20250305",
            "max_uses": 5,
            "allowed_domains": [
                "newyorkfed.org/markets/reference-rates/sofr",
                "fred.stlouisfed.org/series/SOFR"],
            "user_location": {
                "type": "approximate",
                "city": "New York",
                "region": "New York",
                "country": "US",
                "timezone": "America/New_York",
            },
        },
    ]
)

# Print the full response
print("\nFull response:")
print(message.model_dump_json(indent=2))

# Extract and print the content
print("\nResponse content:")
for content_block in message.content:
    if content_block.type == "text":
        print(content_block.text)

# Print usage information
print("\nUsage statistics:")
print(f"Input tokens: {message.usage.input_tokens}")
print(f"Output tokens: {message.usage.output_tokens}")
if message.usage.server_tool_use:
    print(f"Web search requests: {message.usage.server_tool_use.web_search_requests}")

#
# rate_url_dict = {
#     "SOFR": "newyorkfed.org/markets/reference-rates/sofr",
#     "SONIA": "bankofengland.co.uk/boeapps/database/fromshowcolumns.asp?Travel=NIxSUx&FromSeries=1&ToSeries=50&DAT=RNG&FD=1&FM=Jan&FY=2015&TD=20&TM=Jun&TY=2025&FNY=&CSVF=TT&html.x=181&html.y=42&C=5JK&Filter=N",
# }
#
# for ref_rate, url in rate_url_dict.items():
#     rate_retrieval_w_tool(ref_rate, url)
#     break




Full response:
{
  "id": "msg_01TpfmjnB1SGUhr5PCWNVm74",
  "content": [
    {
      "citations": null,
      "text": "Let me search for the latest SOFR rate.",
      "type": "text"
    },
    {
      "id": "srvtoolu_017z3cABY8oVsttZoi9usAxK",
      "input": {
        "query": "latest SOFR rate Federal Reserve Bank of New York today June 20 2025"
      },
      "name": "web_search",
      "type": "server_tool_use"
    },
    {
      "content": [
        {
          "encrypted_content": "EvkLCioIBBgCIiRiMWY1MjM0ZC03YjM0LTQ2MDgtYWU0NS04ZWE5MGE4YzZlMGMSDHxf93sCxf1x0cVtahoMbKTwVo/4zGdpOdDgIjCXumOnihyaIIAbXVgUsZaF26OTivkXGafd/SgJo61mMmZyLOyTAUI1XAyy8z1ucuwq/ApYyw0DgTT6Uve756Xd2LIOm+9rEa4FYlb5aXdZMf8cJq4cj14CSfMrNHcpZHStbqk0Wc85NBQgedJ4Az2FRsdcuP8HJwtPyT2jR5aSkw3eQoRqesh1RMvNnPtEFXz/M2yXejBaJMeyZEYnqTT/Izi5QJmTXRorLW+iueQd8oYzRguTZ9JJ4tH01Zna7P+LN+sZicK2AmulZQ+bcM/2RtfaUrRw7H9hdcDw/iERofUxQ9cm+g0U7lPq3iwshnzrohKpgt0cChBLhCVOWExpbStTzPUHDu/fGSzQ9GNtQBdDVNjOTbsCBrlY9X9xSa+KtM/qNHUpLl+Aq4I+mD7o